# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
from citipy import citipy

# Import API key
from api_keys import geoapify_key

# Configure gmaps
gmaps.configure(api_key=geoapify_key)

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,kodiak,57.7900,-152.4072,38.71,87,100,8.05,US,1675193515
1,1,hualmay,-11.0964,-77.6139,76.15,70,100,8.41,PE,1675193711
2,2,cape town,-33.9258,18.4232,70.88,79,75,18.41,ZA,1675193223
3,3,puerto ayora,-0.7393,-90.3518,84.16,90,65,4.00,EC,1675193533
4,4,hilo,19.7297,-155.0900,77.29,86,40,5.75,US,1675193504


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_locations = city_data_df[['Lat', 'Lng']].astype(float)
humidity = city_data_df['Humidity'].astype(float)

# Display the map
layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'}

#add heatmap
fig = gmaps.figure(map_type='HYBRID', layout=layout)

heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df['Max Temp'] >= 70) & (city_data_df['Max Temp'] <= 80) & 
                                  (city_data_df['Windspeed'] <= 20) & (city_data_df['Humidity'] <= 30)]
ideal_weather



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
173,173,atar,20.5169,-13.0499,71.17,17,51,5.59,MR,1675193758
176,176,kidal,18.4411,1.4078,70.95,6,22,7.29,ML,1675193759
487,487,ouadda,8.0777,22.4007,74.41,18,5,3.31,CF,1675193856


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
hotel_df['Hotel Name'] = ''

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 50000,
    "rankby": "prominence",
    "types": "lodging",
    "keyword": "hotel",
    "key": geoapify_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get variables type from df
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    params["location"] = f'{lat},{lng}'
        
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {city} hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except:
        print(f"Missing field/result for {city}... skipping.")
        
    print("------------")

/var/folders/js/n4bctg1s3m3cyxcglhsfxmvr0000gn/T/ipykernel_82072/4227860344.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


Closest atar hotel is Hotel al waha.
------------
Missing field/result for kidal... skipping.
------------
Missing field/result for ouadda... skipping.
------------


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
0,173,atar,20.5169,-13.0499,71.17,17,51,5.59,MR,1675193758,Hotel al waha
1,176,kidal,18.4411,1.4078,70.95,6,22,7.29,ML,1675193759,
2,487,ouadda,8.0777,22.4007,74.41,18,5,3.31,CF,1675193856,


In [12]:
%%capture --no-display

# Configure the map plot
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))